In [1]:
import numpy as np 
import pandas as pd
import logging 
from dateutil import parser
#from time import time
from datetime import date, timedelta

input_dir = "X:\\Data_Analytics\\Positions\\Input\\"
master_dir = "X:\\Data_Analytics\\Positions\\Master\\"
log_path = "X:\\Data_Analytics\\Positions\\"
output_dir = "X:\\Data_Analytics\\Positions\\Output\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")




In [ ]:
df = pd.read_excel(input_dir+'FNO_IMIS_Trade Report01NOV2018.xls')

# convert date to datetime format
df['Transaction Date'] = df.apply(lambda row: pd.to_datetime(row['Transaction Date'], format='%d-%b-%Y'), axis=1)
df['Expiry'] = df.apply(lambda row: parser.parse(row['Expiry']) , axis=1)


In [ ]:
expiry_encoders = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
# fetch only near month expiry
expiry_encoders = expiry_encoders[['Date','Expiry1']]
# read master file for FUT only stocks


In [ ]:
df.head(1)

In [ ]:
# fetch current date from poisitoning
current_date = df['Transaction Date'][0]

In [ ]:
# fetch near expiry for current date
near_expiry = expiry_encoders[expiry_encoders['Date']==current_date]['Expiry1'].values[0]

In [ ]:
# filter only near expiry stocks
df = df[df['Expiry'] == near_expiry]


In [ ]:
fut_stocks = pd.read_excel(master_dir+'MasterData.xlsx')

In [ ]:
#Filter only active stocks from master file
df = df[df['Symbol'].isin(fut_stocks['SYMBOL']) ]

In [ ]:
# exclude list
exclude =['KSAM13']
# exclude stocks
df = df[~df['Client Id'].isin(exclude)]

In [ ]:
import numpy as np 
import pandas as pd
import logging 
from dateutil import parser
from time import time
import os
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
from dateutil import parser
import datetime, calendar
import sys
import win32com.client

input_dir = "X:\\Data_Analytics\\Positions\\Input\\"
master_dir = "X:\\Data_Analytics\\Positions\\Master\\"
log_path = "X:\\Data_Analytics\\Positions\\"
output_dir = "X:\\Data_Analytics\\Positions\\Output\\"
order_qty_dir = "X:\\Data_Analytics\\Positions\\Order_quantity\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")


def days():
    date = datetime.datetime.today()
    year = date.year
    month = 11
    num_days = calendar.monthrange(year, month)[1]
    days = [datetime.date(year, month, day) for day in range(1, num_days+1)]
    return days


def order_quantity_loop(day):
    '''Func to loop through order quantities file'''
    for r,d,f in os.walk(order_qty_dir):     
        
        logging.info('Look for order quantity file...')
        for excel_file in f:     
            ordr_date = parser.parse(excel_file[3:5]+' '+ excel_file[5:8]+' '+ excel_file[8:-5]).date()   
            logging.info('Order quantity was recieved for day : {0}'.format(ordr_date))
            if day==ordr_date:
                order_qty = pd.read_excel(order_qty_dir+excel_file)
                order_qty = order_qty[['underlying RIC','Pos Current Qty']]           
                order_qty.fillna(0,inplace=True)            
                order_qty.rename(columns= {'Pos Current Qty': 'Ordered_qty_{0}'.format(ordr_date)}, inplace=True)

                return order_qty
                

            
        

def input_file_loop(day):
    pass
    
        
            

def pre_process(excel_file,expiry_encoders,fut_stocks,exclude):
    '''func to pre process input files'''
    
    df = pd.read_excel(input_dir+excel_file)
    # convert date to datetime format
    df['Transaction Date'] = df.apply(lambda row: pd.to_datetime(row['Transaction Date'], format='%d-%b-%Y'), axis=1)
    df['Expiry'] = df.apply(lambda row: parser.parse(row['Expiry']) , axis=1)
    
    # fetch only near month expiry
    expiry_encoders = expiry_encoders[['Date','Expiry1']]
    
    # fetch current date from poisitoning
    current_date = df['Transaction Date'][0]
    # fetch near expiry for current date
    try:
        near_expiry = expiry_encoders[expiry_encoders['Date']==current_date]['Expiry1'].values[0]
        logging.info('Near Expiry for current month is {0}'.format(near_expiry))
    except Exception as e:
        logging.info('Expiry master has no near expiry month for current date {0}'.format(current_date))
    # filter only near expiry stocks
    df = df[df['Expiry'] == near_expiry]
    
    #Filter only active stocks from master file
    df = df[df['Symbol'].isin(fut_stocks['SYMBOL']) ]
    logging.info('{0} Stocks were excluded, since not present in master file'.format(df[~df['Symbol'].isin(fut_stocks['SYMBOL']) ]))
    # exclude stocks from exclude list
    df = df[~df['Client Id'].isin(exclude)]
    
    # Convert SELL stocks to negative quantity
    df['Traded lots'] = df.apply(lambda row: -1*row['Traded lots'] if row['Buy/Sell']=='SELL' else row['Traded lots'], axis=1)
    
    df['Transaction Date'] = df.apply(lambda row: pd.datetime.date(row['Transaction Date']), axis=1)
    df['Expiry'] = df.apply(lambda row: pd.datetime.date(row['Expiry']), axis=1)
    df = df[['Client Id','Symbol','Expiry','Traded lots','Quantity','Transaction Date']]
    # get underlying ric and merge from master file mapping 
    fut_stocks = fut_stocks[['SYMBOL','underlying RIC']]
    fut_stocks.rename(columns= {'SYMBOL':'Symbol'}, inplace = True)
    
    df = df.merge(fut_stocks,on='Symbol',how='left') 
    
    
    print len(df)
    return df


def main():
    '''Main function to pre-process and generate pivot like dataframe'''
    
    logging.info('Reading master files')
    # read expiry contracts master
    expiry_encoders = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
    # read master file for future active stocks
    fut_stocks = pd.read_excel(master_dir+'MasterData.xlsx')
    fut_stocks = fut_stocks[fut_stocks['IsActiveFNO']==True]
    # exclude list
    exclude =['KSAM13']
    result = pd.DataFrame()
    
    # Pre-process all the files in input dir
    for r,d,f in os.walk(input_dir):        
        
        logging.info('Traverse through each input file')
        for excel_file in f:           
            logging.info('pre-process file: {0}'.format(excel_file))
            df = pre_process(excel_file,expiry_encoders,fut_stocks,exclude)
            # append all the pre-procesed files || Warning: Append doesnt happen in place
            result = result.append(df)      
    
        
    
        if len(set(result['Transaction Date'])) == 1:
            done_till_date = result['Transaction Date'][0]
        elif len(set(result['Transaction Date'])) > 1:
            done_till_date = sorted(result['Transaction Date'][0], reverse=True)[0]

        result = result.pivot_table(values=['Traded lots'], index = ['Symbol','underlying RIC'] ,
                                    columns = ['Transaction Date'],fill_value = 0,margins=True,
                                    aggfunc={'Traded lots': np.sum})   


        result.drop('All', axis=0, inplace=True)
        result.rename(columns = {'All':'Done_till_{0}'.format(done_till_date)}, inplace= True)
        result.columns  = result.columns.droplevel(level=0)  

        #result.columns = ['_'.join(col) for col in res]
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter(output_dir+'Report_{0}.xlsx'.format(done_till_date), engine='xlsxwriter')
        result.to_excel(writer, sheet_name='report')       
    
    
    order_quan_df = pd.DataFrame()
    

    order_quan_df.fillna(0,inplace=True)
    order_res = {}
    for key in order_quan_df['underlying RIC']:
        order_res[key]=np.sum(order_quan_df[order_quan_df['underlying RIC']==key]['Ordered_qty'])

        order_quan_df = pd.DataFrame(order_res.items(), columns=['underlying RIC','Ordered_qty'])
        
    
    df = result.merge(order_quan_df, on='underlying RIC', how='right')
    df.fillna(0,inplace=True) 
    
    df['Pending'] = df.iloc[:,-1] - df.iloc[:,-2]
    
    
    df.to_excel('tessss.xlsx')       
            
            
    
'''
    # Get access to the workbook and sheet
    workbook = writer.book
    worksheet = writer.sheets['report']

    column_num = len(result.columns)


    worksheet.write(0, column_num, 'Pending')

    for row in range(1, len(result.index)):
        # Determine where we will place the formula
        cell_location = xl_rowcol_to_cell(row, column_num)
        # Get the range to use for the sum formula
        start_range = xl_rowcol_to_cell(row, column_num-1)
        end_range = xl_rowcol_to_cell(row, 2)
        # Construct and write the formula
        formula = "={0}-{1}".format(start_range,end_range)
        worksheet.write_formula(cell_location, formula)
        


    writer.save()
  

    #result.to_excel('temp.xlsx')
           
start_time = time()

result = main()

end_time = time()

logging.info('Time taken to process :'.format(end_time - start_time))
print "Execution time: {0} Seconds.... ".format(end_time - start_time)
            
result = pd.DataFrame()
for day in days:
    
    order_file = '''
            

In [ ]:
final = pd.read_excel('temp.xlsx')

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('fancy.xlsx', engine='xlsxwriter')
result.to_excel(writer, index=False, sheet_name='report')

# Get access to the workbook and sheet
workbook = writer.book
worksheet = writer.sheets['report']

column_num = len(result.columns)


worksheet.write(1, column_num, 'Pending')

for row in range(3, len(result.index)):
    # Determine where we will place the formula
    cell_location = xl_rowcol_to_cell(row, column_num)
    # Get the range to use for the sum formula
    start_range = xl_rowcol_to_cell(row, column_num-1)
    end_range = xl_rowcol_to_cell(row, 1)
    # Construct and write the formula
    formula = "={0}-{1}".format(start_range,end_range)
    worksheet.write_formula(cell_location, formula)


writer.save()

In [ ]:
final.columns

In [ ]:
result.columns  = result.columns.droplevel(level=0)

In [ ]:
for r,d,f in os.walk(order_qty_dir):        
        
        logging.info('Look for order quantity file...')
        for excel_file in f:           
            logging.info('Order quantity was recieved for day : {0}'.format(excel_file[3:-5]))
            order_qty = pd.read_excel(order_qty_dir+excel_file)
            order_qty = order_qty[['underlying RIC','Pos Current Qty']]
            order_qty.fillna(0,inplace=True)
            print order_qty

In [ ]:
report_df = pd.read_excel(output_dir+'Report_2018-11-15.xlsx',)


In [ ]:
order_quan[order_quan['underlying RIC']=='ABUJ.NS']

In [ ]:
fut_stocks = pd.read_excel(master_dir+'MasterData.xlsx')
fut_stocks = fut_stocks[fut_stocks['IsActiveFNO']==True]

In [12]:
import numpy as np 
import pandas as pd
import logging 
from dateutil import parser
from time import time
import os
import xlsxwriter
from xlsxwriter.utility import xl_rowcol_to_cell
from dateutil import parser
import datetime, calendar
import sys
import win32com.client
from datetime import date, timedelta

input_dir = "X:\\Data_Analytics\\Positions\\Input\\"
master_dir = "X:\\Data_Analytics\\Positions\\Master\\"
log_path = "X:\\Data_Analytics\\Positions\\"
output_dir = "X:\\Data_Analytics\\Positions\\Output\\"
order_qty_dir = "X:\\Data_Analytics\\Positions\\Order_quantity\\"

# log events in debug mode 
logging.basicConfig(filename=log_path+"test.log",
                        level=logging.DEBUG,
                        format="%(asctime)s:%(levelname)s:%(message)s")



logging.info('Reading master files')
# read expiry contracts master
expiry_encoders = pd.read_excel(master_dir+'Expiry_contract_Master.xlsx')
# convert date to datetime format
expiry_encoders['Expiry1'] = expiry_encoders.apply(lambda row: pd.to_datetime(row['Expiry1'], format='%d-%b-%Y'), axis=1)

# read master file for future active stocks
fut_stocks = pd.read_excel(master_dir+'MasterData.xlsx')
fut_stocks = fut_stocks[fut_stocks['IsActiveFNO']==True]
#futstk only stocks
fut_stocks = fut_stocks[fut_stocks['Type']=='SSF']
# exclude list
exclude = pd.read_excel(master_dir+'IgnoreNames.xlsx')
exclude = list(exclude['Exclude'].values)
    
result = fut_stocks[['SYMBOL','underlying RIC']]

done_till_now = result[['underlying RIC']]

total_df = done_till_now
#initialize done till now with zeros
done_till_now['Done_till'] = 0

total_df['Total_qty'] = 0
total_df['Done_till_now'] = 0

# date to keep track of last traded date
last_traded_date = ''
last_order_file = ''
# get calendar days for current month
start_date=datetime.datetime.today().replace(day=1).date() - datetime.timedelta(days=5)
end_date = (datetime.datetime.today().replace(day=1).date() + datetime.timedelta(days=32)).replace(day=1) - datetime.timedelta(days=1)
days = days_func(start_date,end_date)

for day in days:
    
    # check for order files first
    order_file = ml_file_picker(day)
    if order_file!= None:     
        last_order_file = order_file
        # merge donetillnow
        # done till now is calculated in FNO file processor if condition
        # get last traded column
      
        result['Done_till_{0}'.format(last_traded_date)] = done_till_now['Done_till']
        
        total_df['Done_till_now'] += result['Done_till_{0}'.format(last_traded_date)]
        # flush done till now 
        done_till_now['Done_till']=0
        
        
        # crack password for excel file 
        order_qty = password_cracker(order_file)
           
        order_qty = order_qty[['underlying RIC','Pos Current Qty']]
        order_qty.fillna(0,inplace=True)
        order_qty.rename(columns={'Pos Current Qty':'Order_qty_{0}'.format(day)}, inplace=True)
        # merge order quantity in result
        result = result.merge(order_qty, on = 'underlying RIC', how='left')
        result.fillna(0,inplace=True)       
        

    
    
    file_fno = fno_file_picker(day)
    if file_fno!= None:  
                
        # pre-process file 
        df = pre_process(file_fno,expiry_encoders,fut_stocks,exclude)
        
        df = df[['underlying RIC','Traded lots']]
        # colapse same row by addition of traded lots
        df = df.replace('None','').groupby('underlying RIC',as_index=False).agg(np.sum)
        
        # rename to current day
        df.rename(columns={'Traded lots':'{0}'.format(day)}, inplace=True)
        
        result = result.merge(df, on = 'underlying RIC', how='left')
        result.fillna(0, inplace=True)
        
        # add traded lots to done till now
        done_till_now = done_till_now.merge(df, on='underlying RIC', how='left')
        done_till_now.fillna(0, inplace=True)
        
        done_till_now['Done_till'] = done_till_now['Done_till']+done_till_now['{0}'.format(day)]
        done_till_now.drop(columns=['{0}'.format(day)], inplace=True, axis=1) 
        # keep track of last traded date
        last_traded_date = day
        
        # pending based on short term done till now and current order quantity
        # take absolute difference
        try:
            result['pending_on_{0}'.format(day)] =  result.iloc[:,-2] - result.iloc[:,-1]
        except:
            pass
        
    
    #result['pending_on_{0}'.format(day)] = result['Order_qty_{0}'.format(day)] - result['Done_till_{0}'.format(day)]
    #print 'order',len(done_till_now), len(result) 
        
        
result.to_excel(output_dir+'Report.xlsx')   

last_order_df = password_cracker(last_order_file)


            

X:\Anaconda\lib\site-packages\pandas\core\frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


WARNING *** file size (11884) not 512 + multiple of sector size (512)


AttributeError: Excel.Application.Workbooks

In [11]:
def pre_process(excel_file,expiry_encoders,fut_stocks,exclude):
    '''func to pre process input files'''
    
    df = pd.read_excel(input_dir+excel_file)
    # convert date to datetime format
    df['Transaction Date'] = df.apply(lambda row: pd.to_datetime(row['Transaction Date'], format='%d-%b-%Y'), axis=1)
    df['Expiry'] = df.apply(lambda row: parser.parse(row['Expiry']) , axis=1)
    
    # fetch only near month expiry
    expiry_encoders = expiry_encoders[['Date','Expiry1']]
    
    # fetch current date from poisitoning
    current_date = df['Transaction Date'][0]
  
    # fetch near expiry for current date
   
    try:
        near_expiry = pd.to_datetime(expiry_encoders[expiry_encoders['Date']==current_date]['Expiry1'].values[0])  
     
        df = df[df['Expiry'] == near_expiry]
        
        logging.info('Near Expiry for current month is {0}'.format(near_expiry))
    except Exception as e:
        print e
        logging.info(e)
        logging.info('Expiry master has no near expiry month for current date {0}'.format(current_date))
    # filter only near expiry stocks
    
    
    
    #Filter only active stocks from master file
    df = df[df['Symbol'].isin(fut_stocks['SYMBOL']) ]
    logging.info('{0} Stocks were excluded, since not present in master file'.format(df[~df['Symbol'].isin(fut_stocks['SYMBOL']) ]))
    # exclude stocks from exclude list
    df = df[~df['Client Id'].isin(exclude)]
    
    # Convert SELL stocks to negative quantity
    df['Traded lots'] = df.apply(lambda row: -1*row['Traded lots'] if row['Buy/Sell']=='SELL' else row['Traded lots'], axis=1)
    
    
    
    df = df[['Client Id','Symbol','Expiry','Traded lots','Quantity','Transaction Date']]
    # get underlying ric and merge from master file mapping
    fut_stocks = fut_stocks[['SYMBOL','underlying RIC']]
    fut_stocks.rename(columns= {'SYMBOL':'Symbol'}, inplace = True)
    
    df = df.merge(fut_stocks,on='Symbol',how='left')

    
    return df



def fno_file_picker(day):
    
    for r,d,f in os.walk(input_dir):   
        
        for excel_file in f:  
            if excel_file.startswith('FNO_'):
                processing_date = parser.parse(excel_file[21:23]+' '+excel_file[23:26]+' '+excel_file[26:-4]).date()
                if day==processing_date:
                    return excel_file
                
def ml_file_picker(day):
    
    for r,d,f in os.walk(input_dir):  
        
        for excel_file in f:  
                
            if excel_file.startswith('ML_'):
                processing_date = parser.parse(excel_file[3:5]+' '+excel_file[5:8]+' '+excel_file[8:-4]).date()
                if day==processing_date:
                    return excel_file   
                
                
def operator(a,b):
    '''Operator function for pending calculation'''
    # take obsolute difference 
    return abs(a)-abs(b) 
    
def days_func(d1,d2):
    '''Func to get date range to loop over'''   
    days = [d1 + timedelta(days=x) for x in range((d2-d1).days + 1)]
    return days 

def password_cracker(filename):
    '''Func to crack excel password'''
    #win32com to crack password 
    xlApp = win32com.client.Dispatch("Excel.Application")
    xlwb = xlApp.Workbooks.Open(input_dir+filename, True, True, None, Password= 'ML123')
    # xlwb = xlApp.Workbooks.Open(filename)
    xlws = xlwb.Sheets(1) # counts from 1, not from 0
    content = xlws.Range("A:D").Value 

    # Transfer content to pandas dataframe
    dataframe = pd.DataFrame(list(content))

    dataframe.dropna(how='all', inplace=True)
    # create header
    dataframe.columns = dataframe.iloc[0]
    dataframe = dataframe.reindex(dataframe.index.drop(0))
    
    return dataframe  

In [190]:
df = pd.read_excel('Report.xlsx', sheet_name=0)

In [191]:
d = [ x for x in list(df.columns.values.astype(str)) if x.startswith('20')] 
d.append('SYMBOL')
d.append('underlying RIC')
d.append('pending_on_{0}'.format(last_traded_date))
d = d[-3:]+d[:-3]
df = df.reindex(columns = d) # get all the order excuted columns

df['Net Done'] = df.iloc[:, 3:].sum(axis=1, skipna=True)
df = df[['SYMBOL','underlying RIC','Net Done','pending_on_{0}'.format(last_traded_date)]]
df['Order size'] = df['pending_on_{0}'.format(last_traded_date)] - df['Net Done']

SYMBOL underlying RIC  Net Done  pending_on_2
0           ACC         ACC.NS         0           NaN
1      ADANIENT        ADEL.NS         0           NaN
2    ADANIPORTS        APSE.NS         0           NaN
3    ADANIPOWER        ADAN.NS         0           NaN
4    AJANTPHARM        AJPH.NS         0           NaN
5          ALBK        ALBK.NS       -19           NaN
6    AMARAJABAT        AMAR.NS         0           NaN
7     AMBUJACEM        ABUJ.NS         0           NaN
8    APOLLOHOSP        APLH.NS         0           NaN
9    APOLLOTYRE        APLO.NS         0           NaN
10       ARVIND        ARVN.NS         0           NaN
11     ASHOKLEY        ASOK.NS        -3           NaN
12   ASIANPAINT        ASPN.NS      -278           NaN
13   AUROPHARMA        ARBN.NS         0           NaN
14     AXISBANK        AXBK.NS     -1025           NaN
15   BAJAJ-AUTO        BAJA.NS        -1           NaN
16   BAJFINANCE        BJFN.NS       -55           NaN
17   BANKBARODA         BOB.NS        -8           NaN
18    BANKINDIA         BOI.NS         0           NaN
19    BATAINDIA        BATA.NS         0           NaN
20          BEL        BAJE.NS         0           NaN
21         BEML        BEML.NS         0           NaN
22    BHARATFIN        BHAF.NS         0           NaN
23   BHARATFORG        BFRG.NS         0           NaN
24   BHARTIARTL        BRTI.NS     -1074           NaN
25         BHEL        BHEL.NS         0           NaN
26       BIOCON        BION.NS         0           NaN
27     BOSCHLTD        BOSH.NS         0           NaN
28         BPCL        BPCL.NS         0           NaN
29    BRITANNIA        BRIT.NS       -22           NaN
..          ...            ...       ...           ...
179  BALKRISIND        BLKI.NS         0           NaN
180        NBCC        NBCC.NS         0           NaN
181         MGL        MGAS.NS         0           NaN
182  BERGEPAINT        BRGR.NS         0           NaN
183  CANFINHOME        CNFH.NS         0           NaN
184    CHOLAFIN        CHLA.NS         0           NaN
185        GSFC        GSFC.NS         0           NaN
186         MCX        MCEI.NS         0           NaN
187   NESTLEIND        NEST.NS         0           NaN
188      VGUARD        VGUA.NS         0           NaN
189  GODFRYPHLP        GDFR.NS         0           NaN
190  BAJAJFINSV        BJFS.NS       -76           NaN
191        MRPL        MRPL.NS         0           NaN
192     RAYMOND        RYMD.NS         0           NaN
193    RAMCOCEM        TRCE.NS         0           NaN
194  NATIONALUM        NALU.NS        57           NaN
195     RBLBANK        RATB.NS         0           NaN
196  CHENNPETRO        CHPC.NS         0           NaN
197    SREINFRA        SREI.NS         0           NaN
198  MANAPPURAM        MNFL.NS         0           NaN
199  ICICIPRULI        ICIR.NS         0           NaN
200   REPCOHOME        RHFL.NS         0           NaN
201  KAJARIACER        KAJR.NS         0           NaN
202   BANKNIFTY   BANKNIFTY.NS         0           NaN
203       NIFTY          .NSEI         0           NaN
204  NIFTYINFRA              0         0           NaN
205     NIFTYIT              0         0           NaN
206  NIFTYMID50              0         0           NaN
207    NIFTYPSE              0         0           NaN
208   NIFTYCPSE              0         0           NaN

[209 rows x 4 columns]

In [ ]:
    
    # merge on underlying RIC
    last_order_df = last_order_df.merge(df, on='underlying RIC',how='left')
    last_order_df['Order size'] = last_order_df['pending_on_{0}'.format(last_traded_date)] - last_order_df['Net Done']
    
    last_order_df.to_excel(writer, sheet_name='Client_Report{0}'.format(last_traded_date))   
    # save the file 
    writer.close()
    
    # move to email dir for auto emails 
    shutil.copy(output_dir+'Report{0}.xlsx'.format(last_traded_date), email_dir+'Report.xlsx')

In [2]:
df = pd.read_excel('Report.xlsx', sheet_name="Order_summary")

In [17]:
df[df.iloc[:, 2:].sum(axis=1)!=0]

SYMBOL underlying RIC  Net Done  pending_on_2019-01-24  Order size
1      ADANIENT        ADEL.NS         0                    -53         -53
2    ADANIPORTS        APSE.NS         0                    177         177
5          ALBK        ALBK.NS       -20                    120         140
6    AMARAJABAT        AMAR.NS         0                     -4          -4
7     AMBUJACEM        ABUJ.NS         0                    272         272
11     ASHOKLEY        ASOK.NS       -18                    351         369
12   ASIANPAINT        ASPN.NS      -623                    175         798
13   AUROPHARMA        ARBN.NS         0                    340         340
14     AXISBANK        AXBK.NS     -1502                    282        1784
15   BAJAJ-AUTO        BAJA.NS       -24                    415         439
16   BAJFINANCE        BJFN.NS       -71                    742         813
17   BANKBARODA         BOB.NS        -9                    248         257
18    BANKINDIA         BOI.NS         0                     22          22
20          BEL        BAJE.NS         0                      6           6
22    BHARATFIN        BHAF.NS         0                    -34         -34
23   BHARATFORG        BFRG.NS         0                    108         108
24   BHARTIARTL        BRTI.NS     -1074                      7        1081
25         BHEL        BHEL.NS         0                     95          95
27     BOSCHLTD        BOSH.NS        -1                    140         141
28         BPCL        BPCL.NS         0                    114         114
29    BRITANNIA        BRIT.NS       -33                    328         361
30     CADILAHC        CADI.NS         0                    134         134
31        CANBK        CNBK.NS       -12                    162         174
36        CIPLA        CIPL.NS       -29                    421         450
37    COALINDIA        COAL.NS         0                    866         866
39       CONCOR        CCRI.NS         0                      8           8
42        DABUR        DABU.NS       -71                    558         629
46     DIVISLAB        DIVI.NS       -11                    242         253
48      DRREDDY        REDY.NS       -25                    446         471
49    EICHERMOT        EICH.NS        -1                    524         525
..          ...            ...       ...                    ...         ...
130        SAIL        SAIL.NS         0                    -21         -21
131        SBIN         SBI.NS     -1220                    562        1782
132     SIEMENS        SIEM.NS         0                    -13         -13
135  SRTRANSFIN        SRTR.NS       -91                    210         301
137   SUNPHARMA         SUN.NS      -387                    539         926
139   SYNDIBANK        SBNK.NS        -2                     42          44
140    TATACHEM        TTCH.NS         0                     75          75
142   TATAELXSI        TTEX.NS         0                   -102        -102
144  TATAMOTORS        TAMO.NS      -618                    283         901
145  TATAMTRDVR       TAMdv.NS         0                      1           1
146   TATAPOWER        TTPW.NS        -2                    117         119
147   TATASTEEL        TISC.NS      -276                    547         823
148         TCS         TCS.NS      -802                    914        1716
149       TECHM        TEML.NS       -32                    504         536
150       TITAN        TITN.NS       -37                    463         500
156  ULTRACEMCO        ULTC.NS       -13                    595         608
157   UNIONBANK        UNBK.NS      -131                    100         231
158         UPL        UPLL.NS       -10                    374         384
159        VEDL        VDAN.NS         0                    339         339
161       WIPRO        WIPR.NS      -388                    433         821
163     YESBANK        YESB.NS      -732                 